In [1]:
library(tidyverse)

# Custom package
library(rutils)

-- Attaching packages ------------------------------------------------------------------------------- tidyverse 1.3.0 --

v ggplot2 3.3.3     v purrr   0.3.4
v tibble  3.0.6     v dplyr   1.0.4
v tidyr   1.1.2     v stringr 1.4.0
v readr   1.4.0     v forcats 0.5.1

-- Conflicts ---------------------------------------------------------------------------------- tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()



In [2]:
dirs <- rutils::get_dev_directories(dev_paths_file = "../dev_paths.txt")
unified_dsets <- c("unified_cervical_data", "unified_uterine_data", "unified_uterine_endometrial_data")

In [3]:
dset_idx <- 1
save_res <- TRUE

In [4]:
gene2id_df <- read_tsv(paste0(dirs$data_dir, "/", unified_dsets[dset_idx], "/hugo2entrez.tsv")) %>%
    mutate(entrez_gene_id = as.character(entrez_gene_id)) %>%
    rename(entrez = entrez_gene_id, geneID = hugo_symbol)
dge_res_df <- read_tsv(paste0(dirs$analysis_dir, "/deg/", unified_dsets[dset_idx], "_DESeq_results.tsv"))


-- Column specification ------------------------------------------------------------------------------------------------
cols(
  hugo_symbol = col_character(),
  entrez_gene_id = col_double()
)



-- Column specification ------------------------------------------------------------------------------------------------
cols(
  geneID = col_character(),
  baseMean = col_double(),
  log2FoldChange = col_double(),
  lfcSE = col_double(),
  stat = col_double(),
  pvalue = col_double(),
  padj = col_double(),
  qval = col_double()
)




In [5]:
set <- "figo"
ea_dir <- paste0(dirs$analysis_dir, "/enrichment/", unified_dsets[dset_idx], "_", set, "_kegg")
ea_files_short <- list.files(ea_dir)
ea_files_long <- list.files(ea_dir, full.names = TRUE)

In [6]:
# ea_dfs <- list.files(ea_dir, full.names = TRUE) %>% purrr::map(read_tsv)
ea_dfs <- list()
for (i in seq_len(length(ea_files_short))) {
    pwy <- str_replace(ea_files_short[i], ".tsv", "")
    ea_dfs[[pwy]] <- read_tsv(ea_files_long[i], col_names = FALSE) %>%
        mutate(pathway = pwy) %>%
        rename(entrez = X1, color = X2) %>%
        mutate(entrez = as.character(entrez)) %>%
        select(-color)
}


-- Column specification ------------------------------------------------------------------------------------------------
cols(
  X1 = col_double(),
  X2 = col_character()
)



-- Column specification ------------------------------------------------------------------------------------------------
cols(
  X1 = col_double(),
  X2 = col_character()
)



-- Column specification ------------------------------------------------------------------------------------------------
cols(
  X1 = col_double(),
  X2 = col_character()
)



-- Column specification ------------------------------------------------------------------------------------------------
cols(
  X1 = col_double(),
  X2 = col_character()
)



-- Column specification ------------------------------------------------------------------------------------------------
cols(
  X1 = col_double(),
  X2 = col_character()
)



-- Column specification -----------------------------------------------------------------------------------------------

# Look at breakdowns

In [7]:
bind_rows(ea_dfs) %>%
    inner_join(gene2id_df, by = "entrez") %>%
    inner_join(dge_res_df, by = "geneID") %>%
    mutate(de_dir = ifelse(log2FoldChange > 0, "UP", "DOWN")) %>%
    select(geneID, entrez, pathway, de_dir) %>%
    filter(pathway == "protein_digestion_and_absorption") %>%
    arrange(de_dir, geneID)

geneID,entrez,pathway,de_dir
<chr>,<chr>,<chr>,<chr>
COL14A1,7373,protein_digestion_and_absorption,DOWN
COL20A1,57642,protein_digestion_and_absorption,DOWN
COL24A1,255631,protein_digestion_and_absorption,DOWN
COL25A1,84570,protein_digestion_and_absorption,DOWN
COL28A1,340267,protein_digestion_and_absorption,DOWN
PRSS1,5644,protein_digestion_and_absorption,DOWN
COL6A5,256076,protein_digestion_and_absorption,UP
COL9A2,1298,protein_digestion_and_absorption,UP
